In [1]:
import sys, os, re; sys.path = (["../src/", "../"] if re.match(r'^(\w\:\\)|(/)', os.getcwd()) else [])+ sys.path 

from typing import Any, Optional, List

import qubx
%qubxd dev 

%load_ext autoreload
%autoreload 2

from qubx import lookup, logger
from qubx.core.strategy import IStrategy, StrategyContext, TriggerEvent, PositionsTracker
from qubx.pandaz.utils import *
import qubx.pandaz.ta as pta

# - - - - - - - - - - - - - - - - - - - - - - - -
from qubx.ta.indicators import sma, ema
from qubx.data.readers import CsvStorageDataReader, MultiQdbConnector, AsTimestampedRecords, AsQuotes, RestoreTicksFromOHLC, AsPandasFrame, RestoreTicksFromOHLC, AsOhlcvSeries
from qubx.core.basics import Deal, Instrument, Order, Position, Signal
from qubx.utils.charting.lookinglass import LookingGlass

from qubx.backtester.simulator import simulate
from qubx.core.metrics import tearsheet
from qubx.trackers.sizers import FixedSizer, FixedRiskSizer
from qubx.core.metrics import chart_signals

 >  [dev] installing cython rebuilding hook

⠀⠀⡰⡖⠒⠒⢒⢦⠀⠀   
⠀⢠⠃⠈⢆⣀⣎⣀⣱⡀  QUBX | Quantitative Backtesting Environment 
⠀⢳⠒⠒⡞⠚⡄⠀⡰⠁         (c) 2024, ver. Dev
⠀⠀⠱⣜⣀⣀⣈⣦⠃⠀⠀⠀ 
        


# Test raw StopOrder

In [2]:
r = CsvStorageDataReader("../tests/data/csv")

In [36]:
instr = lookup.find_symbol("BINANCE.UM", "BTCUSDT")
ohlc = r.read("BTCUSDT_ohlcv_M1", start='2024-01-01', stop='2024-01-15', transform=AsOhlcvSeries())
stream = r.read("BTCUSDT_ohlcv_M1", start='2024-01-01', stop='2024-01-15', transform=RestoreTicksFromOHLC(trades=False, spread=instr.min_tick))

In [37]:
stream[0].mid_price()

42314.0

In [ ]:
LookingGlass([ohlc.pd()]).look(
    ).hline(stream[0].mid_price() + 1000
    ).hline(stream[0].mid_price() - 150
    ).hline(stream[0].mid_price() + 2000
    ).hline(stream[0].mid_price() - 1900
            ).hover()
# plt.axhline(stream[0].mid_price() + 150)

# Signals

In [7]:
from qubx.core.strategy import IPositionGathering, IStrategy, PositionsTracker, StrategyContext, TriggerEvent
from qubx.core.utils import recognize_time, time_to_str

In [3]:
I = lookup.find_symbol("BINANCE.UM", "BTCUSDT")
ohlc = CsvStorageDataReader("../tests/data/csv").read(
    "BTCUSDT_ohlcv_M1", start="2024-01-01", stop="2024-01-15", transform=AsPandasFrame()
)

if 0:
    LookingGlass([ohlc[1100:1300]]).look(
        ).vline("2024-01-01 20:00:00"
        ).hline(43800, c='red'
        ).hline(43654.50, c='green'
        ).hover(h=300)

In [4]:
class GuineaPig(IStrategy):
    tests = {}

    def on_fit(
        self, ctx: StrategyContext, fit_time: str , previous_fit_time: str | None = None
    ):
        self.tests = {recognize_time(k): v for k, v in self.tests.items()}

    def on_event(self, ctx: StrategyContext, event: TriggerEvent) -> List[Signal] | None:
        r = []
        for k in list(self.tests.keys()):
            if event.time >= k:
                r.append(self.tests.pop(k))
                # print(time_to_str(event.time))
        return r

In [8]:
from qubx.trackers.riskctrl import AtrRiskTracker, StopTakePositionTracker

result = simulate(
    {
        "TEST": [
            GuineaPig(tests={"2024-01-01 20:00:00": I.signal(-1, stop=43800)}),
            StopTakePositionTracker(None, None, sizer=FixedRiskSizer(1)),
        ]
    }, {f"BINANCE.UM:BTCUSDT": ohlc}, 10000,
    instruments=[f"BINANCE.UM:BTCUSDT"], subscription=dict(type="ohlc", timeframe="1Min"), trigger="-1Sec", commissions="vip0_usdt",
    debug="DEBUG", start="2024-01-01", stop="2024-01-03",
)
# print(result[0].executions_log)

2024-09-02 13:06:52.452 [ 🐞 ] 2024-01-01 00:00:00 Initiating simulated trading for binance.um for 10000 x 1.0 in USDT...
2024-01-01 00:00:00.000 [ ℹ️ ] SimulatedData.binance.um initialized
2024-01-01 00:00:00.000 [ 🐞 ] Triggering strategy on every 1Min bar after 59S


Simulating:   0%|          | 0/172800.0 [00:00<?, ?s/s]

2024-01-01 00:00:00.000 [ 🐞 ] (StrategyContext) Subscribing to ohlc updates using {'timeframe': '1Min', 'nback': 1} for 
	['BTCUSDT'] 
2024-01-01 00:00:00.000 [ 🐞 ] SimulatedExchangeService :: subscribe :: BTCUSDT :: ohlc
2024-01-01 00:00:00.000 [ 🐞 ] SimulatedExchangeService :: subscribe :: BTCUSDT :: ohlc
2024-01-01 00:00:00.000 [ ℹ️ ] SimulatedExchangeService :: run :: Simulation started at 2024-01-01
2024-01-01 00:00:00.000 [ 🐞 ] Initializing chunks for each loader
2024-01-01 00:00:00.001 [ 🐞 ] [2024-01-01T00:00:00.001000000]: Invoking GuineaPig on_fit('2024-01-01 00:00:00', 'None')
2024-01-01 00:00:00.001 [ 🐞 ] [2024-01-01T00:00:00.001000000]: GuineaPig is fitted
2024-01-01 20:00:59.999 [ 🐞 ] 	 ::: Start tracking Target for GuineaPig 43654.50 -1.000000 BTCUSDT stop: 43800 on BINANCE.UM -> -0.687 at 2024-01-01T20:00:59.999000000 of BTCUSDT take: None stop: 43800
2024-01-01 20:00:59.999 [ 🐞 ] (StrategyContext) sending market sell for 0.687 of BTCUSDT ...
2024-01-01 20:00:59.999 [ 🐞 

In [12]:
result[0].executions_log

,instrument_id,side,filled_qty,price,commissions,commissions_quoted
timestamp,,,,,,
2024-01-01 20:00:59.999,BTCUSDT,sell,-0.687,43654.45,14.995304,USDT
2024-01-01 20:13:59.999,BTCUSDT,buy,0.687,43800.00,6.018120,USDT


In [11]:
result[0].signals_log

,instrument_id,exchange_id,signal,target_position,reference_price,price,take,stop,group,comment,service
timestamp,,,,,,,,,,,
2024-01-01 20:00:59.999,BTCUSDT,BINANCE.UM,-1,-0.687,43654.5,NaN,None,43800.0,GuineaPig,,False
2024-01-01 20:13:36.000,BTCUSDT,BINANCE.UM,0,0.000,43815.0,43800.0,None,NaN,Risk Manager,Stop triggered,False


In [ ]:
# GuineaPig(tests={"2024-01-01 20:00:00": I.signal(-1, stop=43800)})